# 1. 설계
1. 데이터 준비
    - 개념 설명을 위한 하드코딩된 예시 샘플
    - 간단한 문장 및 그에 대한 품사 태그 정의
2. 어휘 사전(Vocabulary) 및 태그 사전(Tab Vocabulary) 구성
    - 토큰 -> 숫자 변환을 위한 word2index, index2word 매핑 생성
    - 품사 태그 - 숫자 매핑(e.g, tag2index, index2tag)
3. Pytorch Dataset & DataLoader 정의
    - Dataset 클래스 활용, 샘플(토큰 시퀀스) 및 정답(품사 태그 시퀀스) 반환
    - DataLoader 활용, 배치(batch) 단위 데이터 처리
4. 모델(Encoder-Only RNN) 설계
    - Embedding Layer: 입력 단어 임베딩 벡터 변환
    - RNN: 시퀀스 정보 인코딩
    - Fully-Connected Layer: RNN의 은닉 상태(hidden state) 출력을 받아 각 시점(time-step)마다 품사 태그 예측
    - 특징: 일반적인 seq2seq 모델(Encoder-Decoder)와 달리, Encoding-only 접근에서는 인코더에서 바로 분류 수행
5. 학습(Training) 루프
    - 손실 함수(loss function): 토큰 단위 교차 엔토로피 손실
    - 옵티마이저(optimizer): Adam, SGD 등
    - Epoch마다 전체 데이터 반복, 예측-손실 계산-역전파-가중치 갱신 수행
6. 추론(Inference) 및 성능 확인
    - 학습된 모델의 샘플 데이터 예측 결과 확인

# 2. 코드 구현 예시

In [1]:
# 필요 라이브러리 호출
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
# 1. 예시 데이터 준비
sentences = [
    ["나는", "밥을", "먹는다"],
    ["너는", "공부를", "한다"],
    ["그녀는", "책을", "읽는다"],
]

pos_tags = [
    ["PRON", "NOUN", "VERB"],
    ["PRON", "NOUN", "VERB"],
    ["PRON", "NOUN", "VERB"],
]

In [ ]:
# 2. Vocabulary 생성
word_list = set([word for sent in sentences for word in sent])
word_list = list(word_list)
word2index = {w: i+1 for i, w in enumerate(word_list)}
index2word = {i: w for w, i in word2index.items()}

tag_list = set([tag for tags in pos_tags for tag in tags])
tag_list = list(tag_list)
tag2index = {t: i for i, t in enumerate(tag_list)}
index2tag = {i: t for t, i in tag2index.items()}

vocab_size = len(word2index) + 1
tag_size = len(tag2index)

print("Word Vocabulary:", word2index)
print("Tag Vocabulary:", tag2index)

Word Vocabulary: {'너는': 1, '책을': 2, '공부를': 3, '나는': 4, '그녀는': 5, '읽는다': 6, '밥을': 7, '한다': 8, '먹는다': 9}
Tag Vocabulary: {'PRON': 0, 'NOUN': 1, 'VERB': 2}


In [6]:
# 3. Dataset 및 DataLoader 생성
def encode_sentence(sentence):
    """
    단어 사전 활용, 문장 -> 숫자 리스트 변환
    예: ["나는", "밥을", "먹는다"] -> [2, 5, 1]
    """
    return [word2index[w] for w in sentence]

def encode_tag(tags):
    """
    태그 사전 활용, 태그 -> 숫자 리스트 변환
    예: ["PRON", "NOUN", "VERB"] -> [1, 0, 2]
    """
    return [tag2index[t] for t in tags]

class PosDataset(Dataset):
    def __init__(self, sentences, pos_tags):
        super(PosDataset, self).__init__()
        self.sentences = sentences
        self.pos_tags = pos_tags

        # 미리 숫자 인코딩
        self.encoded_sentences = [encode_sentence(sent) for sent in sentences]
        self.encoded_tags = [encode_tag(tags) for tags in pos_tags]

    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        # 각 샘플을 (입력 시퀀스, 타깃 시퀀스) 형태 변환
        return (
            torch.tensor(self.encoded_sentences[idx], dtype=torch.long),
            torch.tensor(self.encoded_tags[idx], dtype=torch.long)
        )

dataset = PosDataset(sentences, pos_tags)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=None)

In [7]:
# 4. 모델(Encoder-Only RNN) 정의
class PosTagger(nn.Module):
    """
    RNN(LSTM) 기반의 품사 태깅 모델.
    단순화를 위해 bidirectionl=False, single-layer로 구성
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tag_size):
        super(PosTagger, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, tag_size)

    def forward(self, x):
        """
        x: [batch_size, seq_len] 형태의 단어 인덱스 시퀀스
        """
        # 1) 임베딩
        embedded = self.embedding(x) # [batch_size, seq_len, embedding_dim]

        # 2) LSTM 적용
        #    output: [batch_size, seq_len, hidden_dim]
        #    (h_n, c_n): LSTM의 마지막 히든, 셀 상태(여기서는 사용 X)
        output, (h_n, c_n) = self.lstm(embedded)

        # 시점별 fc 적용, 태그 확률(로짓) 산출
        logits = self.fc(output) # [batch_size, seq_len, tag_size]

        return logits
    
embedding_dim = 8
hidden_dim = 8

model = PosTagger(vocab_size, embedding_dim, hidden_dim, tag_size)

In [8]:
# 5. 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [12]:
# 6. 학습 루프
for epoch in range(100):
    total_loss = 0.0
    for inputs, targets in dataloader:
        # inputs: [batch_size, seq_len]
        # targets: [batch_size, seq_len]

        optimizer.zero_grad()
        outputs = model(inputs) # [batch_size, seq_len, tag_size]

        batch_size, seq_len, _ = outputs.shape
        outputs_reshaped = outputs.view(batch_size * seq_len, -1)
        targets_reshaped = targets.view(-1)

        loss = criterion(outputs_reshaped, targets_reshaped)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/10], Loss: {total_loss:.4f}")

Epoch [1/10], Loss: 1.6460
Epoch [2/10], Loss: 1.5302
Epoch [3/10], Loss: 1.4318
Epoch [4/10], Loss: 1.3755
Epoch [5/10], Loss: 1.2593
Epoch [6/10], Loss: 1.1556
Epoch [7/10], Loss: 1.0485
Epoch [8/10], Loss: 0.9408
Epoch [9/10], Loss: 0.8350
Epoch [10/10], Loss: 0.7382
Epoch [11/10], Loss: 0.6478
Epoch [12/10], Loss: 0.5868
Epoch [13/10], Loss: 0.5215
Epoch [14/10], Loss: 0.4565
Epoch [15/10], Loss: 0.3947
Epoch [16/10], Loss: 0.3535
Epoch [17/10], Loss: 0.2988
Epoch [18/10], Loss: 0.2751
Epoch [19/10], Loss: 0.2291
Epoch [20/10], Loss: 0.2165
Epoch [21/10], Loss: 0.1789
Epoch [22/10], Loss: 0.1729
Epoch [23/10], Loss: 0.1423
Epoch [24/10], Loss: 0.1402
Epoch [25/10], Loss: 0.1270
Epoch [26/10], Loss: 0.1068
Epoch [27/10], Loss: 0.0978
Epoch [28/10], Loss: 0.0960
Epoch [29/10], Loss: 0.0832
Epoch [30/10], Loss: 0.0756
Epoch [31/10], Loss: 0.0702
Epoch [32/10], Loss: 0.0672
Epoch [33/10], Loss: 0.0631
Epoch [34/10], Loss: 0.0614
Epoch [35/10], Loss: 0.0544
Epoch [36/10], Loss: 0.0515
E

In [13]:
def predict_tags(model, sentences):
    """
    주어진 문장(단어 리스트)에 대해 품사 태그 예측 반환
    """
    model.eval()
    with torch.no_grad():
        encoded = torch.tensor([word2index[word] for word in sentences], dtype=torch.long)
        # [1, seq_len]

        encoded = encoded.unsqueeze(0) # [1, seq_len]

        logits = model(encoded) # [1, seq_len, tag_size]
        predictions = torch.argmax(logits, dim=-1) # [1, seq_len]

        pred_tags = [index2tag[idx.item()] for idx in predictions[0]]
        return pred_tags
    
test_sentence = ["그녀는", "밥을", "먹는다"]
predicted = predict_tags(model, test_sentence)
print("Test Sentence:", test_sentence)
print("Predicted POS:", predicted)

Test Sentence: ['그녀는', '밥을', '먹는다']
Predicted POS: ['PRON', 'NOUN', 'VERB']
